Task 01

In [ ]:
from queue import PriorityQueue

class Node:
    def __init__(self, position, parent=None):
        self.position = position
        self.parent = parent
        self.g = 0
        self.h = 0
        self.f = 0  # Total cost (g + h)

    def __lt__(self, other):
        return self.f < other.f

def heuristic(current_pos, goals):
    return min(abs(current_pos[0] - goal[0]) + abs(current_pos[1] - goal[1]) for goal in goals)

def best_first_search_multiple_goals(maze, start, goals):
    rows, cols = len(maze), len(maze[0])

    if maze[start[0]][start[1]] == 1:
        print("Start position is a wall!")
        return None

    remaining_goals = goals.copy()
    start_node = Node(start)
    path_so_far = [start]

    while remaining_goals:
        current_path = find_path_to_nearest_goal(maze, path_so_far[-1], remaining_goals)

        if not current_path:
            return None

        path_so_far.extend(current_path[1:])

        reached_goal = current_path[-1]
        remaining_goals.remove(reached_goal)

    return path_so_far

def find_path_to_nearest_goal(maze, start, goals):
    rows, cols = len(maze), len(maze[0])
    start_node = Node(start)
    start_node.h = heuristic(start, goals)
    start_node.f = start_node.h

    frontier = PriorityQueue()
    frontier.put((start_node.f, start_node))
    visited = {start}

    while not frontier.empty():
        _, current_node = frontier.get()
        current_pos = current_node.position

        if current_pos in goals:
            path = []
            while current_node:
                path.append(current_node.position)
                current_node = current_node.parent
            return path[::-1]

        for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
            new_pos = (current_pos[0] + dx, current_pos[1] + dy)

            if (0 <= new_pos[0] < rows and
                0 <= new_pos[1] < cols and
                maze[new_pos[0]][new_pos[1]] == 0 and
                new_pos not in visited):

                new_node = Node(new_pos, current_node)
                new_node.g = current_node.g + 1
                new_node.h = heuristic(new_pos, goals)
                new_node.f = new_node.h

                frontier.put((new_node.f, new_node))
                visited.add(new_pos)

    return None

maze = [
    [0, 0, 1, 0, 0],
    [0, 0, 0, 0, 1],
    [1, 1, 0, 1, 0],
    [0, 0, 0, 1, 0],
    [1, 1, 0, 0, 0]
]

start = (0, 0)
goals = [(2, 2), (4, 4)]

def print_maze(maze, path=None, goals=None):
    if path is None:
        path = []
    if goals is None:
        goals = []

    for i in range(len(maze)):
        for j in range(len(maze[0])):
            if (i, j) == start:
                print("S", end=" ")
            elif (i, j) in goals:
                print("G", end=" ")
            elif (i, j) in path:
                print("*", end=" ")
            else:
                print(maze[i][j], end=" ")
        print()

print("Initial Maze:")
print_maze(maze, goals=goals)
print("\n")

path = best_first_search_multiple_goals(maze, start, goals)

if path:
    print("Path found:", path)
    print("\nMaze with Path:")
    print_maze(maze, path=path, goals=goals)
else:
    print("No path found")

Initial Maze:
S 0 1 0 0 
0 0 0 0 1 
1 1 G 1 0 
0 0 0 1 0 
1 1 0 0 G 


Path found: [(0, 0), (1, 0), (1, 1), (1, 2), (2, 2), (3, 2), (4, 2), (4, 3), (4, 4)]

Maze with Path:
S 0 1 0 0 
* * * 0 1 
1 1 G 1 0 
0 0 * 1 0 
1 1 * * G 


Task 02

In [ ]:
import random
from queue import PriorityQueue

def astar_dynamic(graph, start, goal, heuristic):
    open_set = PriorityQueue()
    open_set.put((heuristic[start], 0, start))

    came_from = {}
    g_score = {node: float('inf') for node in graph}
    g_score[start] = 0

    counter = 1

    used_costs = {}

    while not open_set.empty():
        _, _, current = open_set.get()

        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            return path[::-1]

        for neighbor, base_cost in graph[current].items():
            dynamic_cost = max(1, base_cost + random.randint(-2, 2))
            used_costs[(current, neighbor)] = dynamic_cost

            tentative_g = g_score[current] + dynamic_cost

            if tentative_g < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g

                f_score = tentative_g + heuristic[neighbor]
                counter += 1
                open_set.put((f_score, counter, neighbor))

    return None

graph = {
    'A': {'B': 2, 'C': 1},
    'B': {'A': 2, 'D': 4, 'E': 3},
    'C': {'A': 1, 'F': 1},
    'D': {'B': 4, 'H': 2},
    'E': {'B': 3, 'H': 5},
    'F': {'C': 1, 'I': 6},
    'G': {'I': 4},
    'H': {'D': 2, 'E': 5, 'I': 3},
    'I': {'F': 6, 'G': 4, 'H': 3}
}

heuristic = {
    'A': 7, 'B': 6, 'C': 5, 'D': 4,
    'E': 7, 'F': 3, 'G': 6, 'H': 2, 'I': 0
}
random.seed(42)

path = astar_dynamic(graph, 'A', 'I', heuristic)

if path:
    print(f"Path found: {' -> '.join(path)}")

    print("\nPath details:")
    for i in range(len(path)-1):
        node1, node2 = path[i], path[i+1]
        cost = graph[node1][node2]
        print(f"{node1} -> {node2} (base cost: {cost})")
else:
    print("No path found")

print("\nRunning 3 more times to show dynamic path selection:")
for i in range(3):
    result = astar_dynamic(graph, 'A', 'I', heuristic)
    if result:
        print(f"Run {i+1}: {' -> '.join(result)}")

Graph with Dynamic Costs:
A: {'B': 2, 'C': 1}
B: {'A': 2, 'D': 4, 'E': 3}
C: {'A': 1, 'F': 1}
D: {'B': 4, 'H': 2}
E: {'B': 3, 'H': 5}
F: {'C': 1, 'I': 6}
G: {'I': 4}
H: {'D': 2, 'E': 5, 'I': 3}
I: {'F': 6, 'G': 4, 'H': 3}
Path found: A -> C -> F -> I

Path details:
A -> C (base cost: 1)
C -> F (base cost: 1)
F -> I (base cost: 6)

Running 3 more times to show dynamic path selection:
Run 1: A -> C -> F -> I
Run 2: A -> B -> D -> H -> I
Run 3: A -> C -> F -> I


Task 03

In [ ]:
def greedy_bfs_time_windows(graph, start, deliveries):
    frontier = [(start, 0)]
    visited = set()
    path = []

    while frontier:
        frontier.sort(key=lambda x: x[1])
        current_node, arrival_time = frontier.pop(0)

        if current_node in visited:
            continue

        visited.add(current_node)
        path.append(current_node)

        if not deliveries:
            break

        if current_node in deliveries:
            delivery_time = deliveries[current_node]
            if arrival_time > delivery_time:
                print(f"Missed delivery at {current_node}")
            else:
                print(f"Delivered at {current_node} on time")
            del deliveries[current_node]

        for neighbor, travel_time in graph[current_node].items():
            if neighbor not in visited:
                frontier.append((neighbor, arrival_time + travel_time))

    return path

graph = {
    'A': {'B': 2, 'C': 1},
    'B': {'D': 4, 'E': 3},
    'C': {'F': 1, 'G': 5},
    'D': {'H': 2},
    'E': {},
    'F': {'I': 6},
    'G': {},
    'H': {},
    'I': {}
}
deliveries = {'C': 5, 'F': 10, 'I': 15}

# Run Greedy BFS
path = greedy_bfs_time_windows(graph, 'A', deliveries)
print("Delivery route:", path)

Delivered at C on time
Delivered at F on time
Delivered at I on time
Delivery route: ['A', 'C', 'B', 'F', 'E', 'G', 'D', 'I', 'H']


Task 04

In [ ]:
def greedy_bfs_traffic(graph, start, goal, traffic_updates):
    frontier = [(start, 0)]
    visited = set()
    path = []

    while frontier:
        frontier.sort(key=lambda x: x[1])
        current_node, travel_time = frontier.pop(0)

        if current_node == goal:
            path.append(current_node)
            return path

        if current_node in visited:
            continue

        visited.add(current_node)
        path.append(current_node)
        for neighbor, base_time in graph[current_node].items():
            updated_time = base_time + traffic_updates.get((current_node, neighbor), 0)
            frontier.append((neighbor, travel_time + updated_time))
            print(f"Updated travel time from {current_node} to {neighbor}: {travel_time + updated_time}")

    return None

graph = {
    'A': {'B': 2, 'C': 1},
    'B': {'D': 4, 'E': 3},
    'C': {'F': 1, 'G': 5},
    'D': {'H': 2},
    'E': {},
    'F': {'I': 6},
    'G': {},
    'H': {},
    'I': {}
}
traffic_updates = {('A', 'B'): 1, ('C', 'F'): 2}

# Run Greedy BFS
path = greedy_bfs_traffic(graph, 'A', 'I', traffic_updates)
print("Emergency route:", path)

Updated travel time from A to B: 3
Updated travel time from A to C: 1
Updated travel time from C to F: 4
Updated travel time from C to G: 6
Updated travel time from B to D: 7
Updated travel time from B to E: 6
Updated travel time from F to I: 10
Updated travel time from D to H: 9
Emergency route: ['A', 'C', 'B', 'F', 'G', 'E', 'D', 'H', 'I']


Task 05

In [ ]:
from queue import PriorityQueue

def astar_delivery(graph, start, deliveries, fuel_capacity):
    open_set = PriorityQueue()
    initial_heuristic = len(deliveries)
    open_set.put((initial_heuristic, start, fuel_capacity, deliveries.copy(), [start]))
    visited = set()

    def heuristic(remaining_deliveries):
        return len(remaining_deliveries)

    step = 1

    while not open_set.empty():
        current_priority, current, fuel, remaining_delivs, current_path = open_set.get()

        print(f"Step {step}:")
        print(f"  - Current Node: {current}")
        print(f"  - Fuel Left: {fuel}")
        print(f"  - Deliveries Remaining: {remaining_delivs}")
        print(f"  - Current Path: {current_path}")

        step += 1
        state = (current, fuel, frozenset(remaining_delivs.keys()))
        if state in visited:
            print("  - State already visited, skipping.\n")
            continue
        visited.add(state)

        if current in remaining_delivs:
            time_window, fuel_needed = remaining_delivs[current]
            current_steps = len(current_path) - 1

            if current_steps <= time_window and fuel >= fuel_needed:
                print(f"  - Delivering at {current} (Fuel Required: {fuel_needed})")
                new_fuel = fuel - fuel_needed
                new_delivs = remaining_delivs.copy()
                del new_delivs[current]

                if not new_delivs:
                    print("  - All deliveries completed!\n")
                    return current_path

                new_priority = len(current_path) + heuristic(new_delivs)
                open_set.put((new_priority, current, new_fuel, new_delivs, current_path))

        for neighbor, cost in graph[current].items():
            if cost <= fuel:
                new_fuel = fuel - cost
                new_path = current_path + [neighbor]
                new_priority = len(new_path) + heuristic(remaining_delivs)

                print(f"  - Exploring Neighbor: {neighbor} (Cost: {cost}, Fuel Left: {new_fuel})")
                open_set.put((new_priority, neighbor, new_fuel, remaining_delivs.copy(), new_path))

        print()

    print("No valid path found.")
    return None

graph = {
    'A': {'B': 2, 'C': 3},
    'B': {'A': 2, 'D': 4},
    'C': {'A': 3, 'D': 2},
    'D': {'B': 4, 'C': 2},
    'E': {'F': 3},
    'F': {'E': 3}
}

deliveries = {
    'B': (5, 2),
    'D': (10, 3)
}
fuel_capacity = 20

path = astar_delivery(graph, 'A', deliveries, fuel_capacity)
print("Optimized delivery route:", path)


Step 1:
  - Current Node: A
  - Fuel Left: 20
  - Deliveries Remaining: {'B': (5, 2), 'D': (10, 3)}
  - Current Path: ['A']
  - Exploring Neighbor: B (Cost: 2, Fuel Left: 18)
  - Exploring Neighbor: C (Cost: 3, Fuel Left: 17)

Step 2:
  - Current Node: B
  - Fuel Left: 18
  - Deliveries Remaining: {'B': (5, 2), 'D': (10, 3)}
  - Current Path: ['A', 'B']
  - Delivering at B (Fuel Required: 2)
  - Exploring Neighbor: A (Cost: 2, Fuel Left: 16)
  - Exploring Neighbor: D (Cost: 4, Fuel Left: 14)

Step 3:
  - Current Node: B
  - Fuel Left: 16
  - Deliveries Remaining: {'D': (10, 3)}
  - Current Path: ['A', 'B']
  - Exploring Neighbor: A (Cost: 2, Fuel Left: 14)
  - Exploring Neighbor: D (Cost: 4, Fuel Left: 12)

Step 4:
  - Current Node: A
  - Fuel Left: 14
  - Deliveries Remaining: {'D': (10, 3)}
  - Current Path: ['A', 'B', 'A']
  - Exploring Neighbor: B (Cost: 2, Fuel Left: 12)
  - Exploring Neighbor: C (Cost: 3, Fuel Left: 11)

Step 5:
  - Current Node: C
  - Fuel Left: 17
  - Deliverie